# Stable Mariages

For this workshop we'll implement a version of the **Nobel Prize-winning algorithm** for the stable marriage problem. It's called the [Gale-Shapley](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) algorithm AKA the "Deferred Acceptance" algorithm.

The Gale Shapley algorithm solves what's called the **Stable Marriage** Problem. Here is the problem framed as marrying men and women: 

> Given $n$ men and $n$ women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.

This little model however is generally applied to more than marrying men and women. For example, it's the algorithm used to match newly graduated doctors with hospitals, and a variation on it matches kidney patients with organ donors in our hospital system.

The algorithmic question is, given lists of preferences as input, can we find a stable marriage? Can we even guarantee a stable marriage will exist for any set of preferences? The answer to both questions is yes, and it uses an algorithm called deferred acceptance.

Here is an informal description of the algorithm. It goes in rounds. In each round, each man “proposes” to the highest-preferred woman that has not yet rejected him. On the other side, each woman holds a reference to a man at all times. If a woman gets new proposals in a round, she immediately rejects every proposer except her most preferred, but does not accept that proposal. She “defers” the acceptance of the proposal until the very end.

![](Gale-Shapley.gif)

# Your Task

Build a solution `gale_shapley(men, women) -> dict` Where the input is a list of suitors and a list of Suiteds with every one in these lists holding their lists of preferences. The output is a dictionary `suitor -> suited`

Here is a possible way to design a solution:

### Man Class

The `Man(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A method `Suitor.preference()` which points to its current possible reference. It should start by pointing at `preference_list[0]` and every time the Suitor gets rejected in the algorithm, this should point to the next preference.

### Woman Class

The `Woman(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A set of current suitors

- A method `Suited.reject()` which returns all current suitors except the most preferred one

### The Stable Mariage Algorithm

Takes in a list of men and women, loops over suitors trying to find a match until there aren't any unassigned suitors left. Here is the pseudocode for the algorithm:

```
algorithm stable_matching
    Initialize all m ∈ M and w ∈ W to free
    while ∃ free man m who still has a woman w to propose to do
        w := first woman on m's list to whom m has not yet proposed
        if w is free then
            (m, w) become engaged
        else some pair (m', w) already exists
            if w prefers m to m' then
                m' becomes free
                (m, w) become engaged 
            else
                (m', w) remain engaged
            end if
        end if
    repeat
```


Here is some example code of a solution working:

```
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

stable_marriage(men, women)
```

output:

```
{0: 0, 
 2: 1, 
 1: 2, 
 3: 3}
```

In [2]:
class Man():
    def __init__(self, id, preference_list):
        '''
        Stores in variable id, an id number for each man
        Stores in a variable preference_list, the preference list of each man
        Stores the index of the most prefered woman amongst the remaining women that have not rejected him.
        '''
        self.id = id
        self.preference_list = preference_list
        self.updated_preference = 0

    def rejected(self):
        '''
        Once a man is rejected, the function updates the index to his next preference within the preference list
        '''
        self.updated_preference +=1
        
    def preference(self):
        '''
        Returns the man's current preference amongst all the remaining women that have not rejected him.
        '''
        return self.preference_list[self.updated_preference]
    
class Woman:
    def __init__(self, id, preference_list):
        '''
        Stores in variable id, an id number for each woman
        Stores in a variable preference_list, the preference list of each woman
        Stores in an array all the current suitors for each woman
        '''
        self.id = id
        self.preference_list = preference_list
        self.current_suitors = []
        
def stable_marriage(men, women):
    
    stable_pairs = {}
    free_men = men.copy()
    updated_free_men = []
    women = women.copy()
    #Loop until each man and woman has found a match
    while(len(free_men) >0):
        all_men = men.copy()
        updated_free_men = []
        
        #Loop for all free men
        for man in free_men:
            #select the given man's preference
            w = women[man.preference()]
            
            #if the woman is not matched then match them together
            if w.id not in stable_pairs.values():
                stable_pairs[man.id] = w.id
                w.current_suitors.insert(0, man.id)
                
            #else the woman is currently matched with another man
            else:
                #if the woman prefers the given man over her previous match then :
                #delete previous match
                #create new match with the givan man
                #update the previous match's preference considering he was rejected
                #store the previous match in an array to track available men after each iteration
                #update the woman's current suitors
                if w.preference_list.index(man.id) < w.preference_list.index(w.current_suitors[0]):
                    del stable_pairs[w.current_suitors[0]]
                    stable_pairs[man.id] = w.id
                    all_men[w.current_suitors[0]].rejected()
                    updated_free_men.append(w.current_suitors[0])
                    w.current_suitors.insert(0, man.id)
                else:
                #else the given man is rejected and is stored in the array to track available men after each iteration
                    updated_free_men.append(man.id)
                    man.rejected()
                    
        #update the array free men            
        free_men = [all_men[n]for n in updated_free_men]
        
    return stable_pairs

In [3]:
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

stable_marriage(men, women)

{0: 0, 1: 2, 2: 1, 3: 3}

In [4]:
men = [
    Man(id=0, preference_list=[0,1,2]),
    Man(id=1, preference_list=[2,0,1]),
    Man(id=2, preference_list=[1,0,2]),
]

women = [
    Woman(id=0, preference_list=[0,1,2]),
    Woman(id=1, preference_list=[2,0,1]),
    Woman(id=2, preference_list=[1,0,2]),
]

stable_marriage(men, women)

{0: 0, 1: 2, 2: 1}

In [5]:
men = [
    Man(id=0, preference_list=[0,4,1,5,2,3]),
    Man(id=1, preference_list=[4,2,5,3,0,1]),
    Man(id=2, preference_list=[4,1,0,3,2,5]),
    Man(id=3, preference_list=[4,3,2,1,0,5]),
    Man(id=4, preference_list=[4,2,3,1,0,5]),
    Man(id=5, preference_list=[0,1,4,2,3,5]),
]

women = [
    Woman(id=0, preference_list=[0,1,4,2,3,5]),
    Woman(id=1, preference_list=[4,2,3,0,1,5]),
    Woman(id=2, preference_list=[5,1,0,3,4,2]),
    Woman(id=3, preference_list=[5,4,3,2,1,0]),
    Woman(id=4, preference_list=[5,3,4,2,1,0]),
    Woman(id=5, preference_list=[0,1,4,2,3,5]),
]

stable_marriage(men, women)

{0: 0, 1: 2, 2: 1, 4: 3, 5: 4, 3: 5}

In [6]:
men = [
    Man(id=0, preference_list=[0,4,1,2,3]),
    Man(id=1, preference_list=[4,2,3,0,1]),
    Man(id=2, preference_list=[4,1,0,3,2]),
    Man(id=3, preference_list=[4,3,2,1,0]),
    Man(id=4, preference_list=[4,2,3,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,4,2,3]),
    Woman(id=1, preference_list=[4,2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,4,2]),
    Woman(id=3, preference_list=[4,3,2,1,0]),
    Woman(id=4, preference_list=[3,4,2,1,0]),
]

stable_marriage(men, women)

{0: 0, 3: 4, 1: 2, 2: 1, 4: 3}